### Selected model is noaug-grad, here are the results.

In [60]:
from torch import nn
import pretrainedmodels
import pandas as pd
import torch
import numpy as np

In [67]:
#load xception model
import pretrainedmodels

model_name = 'xception' # could be fbresnet152 or inceptionresnetv2
model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')
#add last layer with 3 classes
model.last_linear = nn.Sequential(nn.Linear(2048,3))
#put model on gpu
model.cuda()
#load trained model with weights
model.load_state_dict(torch.load('trained_model_noaug-grad.pt'))

<All keys matched successfully>

In [68]:
#create empty pandas data frame with specified colunm names.
results = pd.DataFrame(columns=['Id','task_1','task_2'])


In [80]:
#load test images
from torchvision import datasets
from torch.utils.data.dataloader import DataLoader
from torchvision import transforms
#resize,crop,transfer to pytorch tensor and normalize the images
transform = transforms.Compose([transforms.Resize(299),
                                     transforms.CenterCrop(299),
                                        transforms.ToTensor(),
                                        transforms.Normalize((0.485, 0.456, 0.406), 
                                             (0.229, 0.224, 0.225))])                                        
test_data = datasets.ImageFolder(root = 'data/test',transform = transform)
test_loader = DataLoader(test_data,batch_size = 1,num_workers = 0)
print(test_data.class_to_idx)

{'melanoma': 0, 'nevus': 1, 'seborrheic_keratosis': 2}


In [81]:
#get all pictures id's
import glob
test_dir = glob.glob('data/*/*/*')
#add to pandas dataframe
results['Id'] = test_dir
print(results)

Id task_1 task_2
0                data\test\melanoma\ISIC_0012258.jpg    NaN    NaN
1                data\test\melanoma\ISIC_0012356.jpg    NaN    NaN
2                data\test\melanoma\ISIC_0012369.jpg    NaN    NaN
3                data\test\melanoma\ISIC_0012395.jpg    NaN    NaN
4                data\test\melanoma\ISIC_0012425.jpg    NaN    NaN
..                                               ...    ...    ...
595  data\test\seborrheic_keratosis\ISIC_0014647.jpg    NaN    NaN
596  data\test\seborrheic_keratosis\ISIC_0014648.jpg    NaN    NaN
597  data\test\seborrheic_keratosis\ISIC_0014649.jpg    NaN    NaN
598  data\test\seborrheic_keratosis\ISIC_0014652.jpg    NaN    NaN
599  data\test\seborrheic_keratosis\ISIC_0014653.jpg    NaN    NaN

[600 rows x 3 columns]


In [82]:
#define criterion
criterion = nn.CrossEntropyLoss()


In [92]:
#make predictions

def test(loaders, model, criterion, use_cuda):
    #make list probalities for melanoma (task_1) and seborrheic keratosis (task_2)
    task_1 = []
    task_2 = []
    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders):
        with torch.no_grad():
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            melanoma_proba = torch.topk(output,k = 3,sorted= False)[0].tolist()[0][0]
            seborrheic_keratosis_proba = torch.topk(output,k = 3,sorted= False)[0].tolist()[0][1]
            #create lists of melanoma probablities if its greater then 0, otherwise append a zero to the list
            if melanoma_proba > 0:
                task_1.append(melanoma_proba)
            else:
                task_1.append(0)
            #create lists of melanoma probablities if its greater then 0, otherwise append a zero to the list
            if seborrheic_keratosis_proba > 0:
                task_2.append(seborrheic_keratosis_proba)
            else:
                task_2.append(0)

            

            # calculate the loss
            loss = criterion(output, target)
            # update average test loss 
            test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
            # convert output probabilities to predicted class
            pred = output.data.max(1, keepdim=True)[1]
            # compare predictions to true label
            correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
            total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

# call test function    
test(test_loader, model, criterion, True)

Test Loss: 0.818344


Test Accuracy: 68% (411/600)


In [ ]:
#add those lists as entries to our task_1 and task_2 columns
results['task_1'] = task_1
results['task_2'] = task_2

In [ ]:
#convert the pandas